In [23]:
import torch
import torch.nn as nn
import shutil
import os
from torch.utils.data import Dataset, DataLoader
import unittest
from d2l import torch as d2l
import logging

### Setting up logger

In [11]:
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

### Dataset
###  Process the english spanish translation
### link: https://www.manythings.org/anki/

In [13]:
class SpanishDataset(Dataset):

    def setup_logger(self, level = logging.DEBUG):
        self.logger = logging.getLogger()
        self.logger.setLevel(level)


    def log(self, message:str, level: str = 'debug'):
        if level == 'debug':
            self.logger.debug(message)
        if level == 'info':
            self.logger.info(message)
        if level == 'warning':
            self.logger.warning(message)
        if level == 'error':
            self.logger.error(message)
        

    def __init__(self, debug = False, num_steps = 10):
        super().__init__()

        self.setup_logger()

        self.DATASET_PATH = '../data/translation/spa.txt'
        assert os.path.exists(self.DATASET_PATH), 'English spanish dataset is not found'
        self.num_steps = num_steps

        self.source = []
        self.target = []

        self.log('start building the dataset')

        with open(self.DATASET_PATH, 'r') as file:
            for idx, line in enumerate(file.readlines()):
                processed = self._preprocess(line)
                source_tokens, target_tokens = self._tokenize(processed)
                self.source.append(source_tokens)
                self.target.append(target_tokens)

        self.log(f'done tokenizing source and target, source len = {len(self.source)}, target len = {len(self.target)}', 'info')

        (self.source_array, self.target_array, self.valid_len, self.label_target_array), self.source_vocab, self.target_vocab = \
            self._build_arrays(self.source, self.target)
        
        shape2d = lambda a: f'({len(a)},{len(a[0])})'
        self.log(f'done building source and target arrays', 'info')
        self.log(f'source array shape {shape2d(self.source_array)}') 
        self.log(f'source vocab len = {len(self.source_vocab)}')
        self.log(f'valid_len shape  = {self.valid_len.shape}')
        self.log(f'target array shape = {shape2d(self.target_array)}')
        self.log(f'target vocab len = {len(self.target_vocab)}')
        
    def _preprocess(self, text):
        # from D2L processing step in chapter 10
        # Replace non-breaking space with space
        text = text.replace('\u202f', ' ').replace('\xa0', ' ')
        # Insert space between words and punctuation marks
        no_space = lambda char, prev_char: char in ',.!?' and prev_char != ' '
        out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
            for i, char in enumerate(text.lower())]
        return ''.join(out)
    
    def _tokenize(self, text):
        # Tokenization method in D2L processing step in chapter 10
        if len(text.split('\t')[:-1]) == 2:
            part = text.split('\t')[:-1]
            src = [token for token in f'{part[0]} <eos>'.split(' ') if token]
            tgt = [token for token in f'{part[1]} <eos>'.split(' ') if token]
            return src, tgt
        else:
            return '',''
    def _build_arrays(self, source, target):
        '''
        @params:
            source_raw: list[list[string]], source sequence, eg: [['a', 'b', '<eos>'], ...]
            target_rwa: list[list[string]], target sequence
        @return
            (
                source_array: list[list[int]]
                target_array_with_bos: list[list[int]]
                valid_len: list[int]
                target_array_with_eos: list[list[int]]
            ),
            source_vocab: Vocab
            target_vocab: Vocab
        '''
        #pad with <pad> token if sequence len < time step, else truncate
        #NOTE: in the book, they just truncated without adding <eos> at the end, 
        # I don't think that is correct
        pad_or_truncate = lambda sentence, numstep: \
            sentence[:numstep - 1] + ['<eos>'] if len(sentence) > numstep \
                else sentence + ['<pad>'] * (numstep - len(sentence))

        def _build_array(sequence, is_target = False):
            '''
            @params:
                sentence: string
                is_target: boolean, if sentence is target, append <bos> to beginning of sentence
            @return
                array: list[str] 
                vocab: Vocab object
            '''
            new_sequence = [ ]
            for sentence in sequence:
                sentence = pad_or_truncate(sentence, self.num_steps)
                if is_target: 
                    sentence = ['<bos>'] + sentence
                
                new_sequence.append(sentence)

            vocab = d2l.Vocab(new_sequence, min_freq = 2)

            #calculate valid_len for training later
            array = torch.tensor([vocab[sentence] for sentence in new_sequence])
            valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
            return array,vocab,valid_len

        source_array, source_vocab, valid_len = _build_array(source)        
        target_array, target_vocab, _ = _build_array(target, is_target= True)        

        return (source_array, target_array[:,:-1], valid_len, target_array[:,1:]), source_vocab, target_vocab
    
    def __len__(self):
        '''
        @return
            int: length of the english - spanish pairs
        '''
        return len(self.source_array)
    
    def __getitem__(self,idx):
        '''
        @params:
            idx: int, datapoint index
        @return
            source_array, target_array, valid_len, label_target_array
        '''
        return (self.source_array[idx], self.target_array[idx], self.valid_len[idx], self.label_target_array[idx])
    
    def get_dataloader(self, batch_size = 32):
        return DataLoader(self, batch_size = batch_size, shuffle = True)
    
dataset = SpanishDataset()

DEBUG - start building the dataset
INFO - done tokenizing source and target, source len = 141370, target len = 141370
INFO - done building source and target arrays
DEBUG - source array shape (141370,10)
DEBUG - source vocab len = 9538
DEBUG - valid_len shape  = torch.Size([141370])
DEBUG - target array shape = (141370,10)
DEBUG - target vocab len = 16679


In [3]:
print(dataset.source[1000])
source = dataset.source
target = dataset.target

['put', 'it', 'on', '.', '<eos>']


In [6]:
print(dataset.source_array[0])
print(dataset.source_vocab.to_tokens(dataset.source_array[500].tolist()))

tensor([3825,   84,  201,  202,  202,  202,  202,  202,  202,  202])
["i'm", 'warm', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [22]:
train_dataloader = dataset.get_dataloader()
sample = next(iter(train_dataloader))

for data in sample:
    print(f'shape = {data.shape},\t data = {data[0]}')

shape = torch.Size([32, 10]),	 data = tensor([4370, 5115, 3555, 8491, 4793,   84,  201,  202,  202,  202])
shape = torch.Size([32, 10]),	 data = tensor([ 189, 6750, 2399, 9021, 9360,   78,  190,  191,  191,  191])
shape = torch.Size([32]),	 data = 7
shape = torch.Size([32, 10]),	 data = tensor([6750, 2399, 9021, 9360,   78,  190,  191,  191,  191,  191])


### Encoder and Decoder Architecture

### Encoder

In [62]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, embed_size, num_layers, num_hiddens, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(input_size = embed_size, num_layers = num_layers, hidden_size = num_hiddens, dropout = dropout)
        #custom initialization

    def forward(self, x, *args):
        #why x.t(), still confused in the book
        emb = self.embedding(x.t())
        output, state = self.rnn(emb)
        return output, state


In [63]:
# Test with sample
vocab_size, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
batch_size, num_steps = 32, 10

x = torch.randint(0,vocab_size,(batch_size, num_steps))

encoder = Encoder(vocab_size, embed_size, num_layers, num_hiddens)

output, state = encoder(x)

print('output shape = ',output.shape, ' dtype = ', output.dtype)
print('state shape = ', state.shape, ' dtype = ', state.dtype)

output shape =  torch.Size([10, 32, 16])  dtype =  torch.float32
state shape =  torch.Size([2, 32, 16])  dtype =  torch.float32


### Decoder

In [103]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout = 0):
        #note that the vocab size in decoder is target language vocab size, not source language vocab size
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(input_size = embed_size + num_hiddens, num_layers = num_layers, hidden_size = num_hiddens, dropout = dropout)
        self.dense = nn.LazyLinear(vocab_size)
        #custom init module
    
    def init_state(self, enc_output, *args):
        '''
        use encoder's output to initialize state
        @params:
            encoder_output
        @return:
            decoder_input
        '''
        return enc_output
    
    def forward(self, x, state):
        emb = self.embedding(x.t())

        enc_output, enc_state = state
        #context variable 
        context = enc_output[-1]
        context = context.repeat(emb.shape[0], 1, 1)
        emb_and_context = torch.cat((emb, context), -1)
        dec_output, dec_state = self.rnn(emb_and_context, enc_state)

        #pass to dense layer and swap back (batch_size, num_steps)
        y_pred = self.dense(dec_output).swapaxes(0,1)
        # print(x.shape)
        # print('embedding shape = ', emb.shape)
        # print('context shape = ', context.shape)
        # print('emb and context shape = ', emb_and_context.shape)
        # print('decoder output shape = ', output.shape)
        # print('decoder state shape = ', dec_state.shape)
        return y_pred, (dec_output, dec_state)



In [104]:
# Test with sample
vocab_size, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
batch_size, num_steps = 32, 10

x = torch.randint(0,vocab_size,(batch_size, num_steps))

encoder = Encoder(vocab_size, embed_size, num_layers, num_hiddens)

enc_output, enc_state = encoder(x)

decoder = Decoder(vocab_size, embed_size, num_hiddens, num_layers)
decoder_state = decoder.init_state((enc_output, enc_state))

y, (dec_output, dec_state) = decoder(x, decoder_state)

print('y_pred shape = ', y.shape)
print('dec state shape  = ', dec_state.shape)


y_pred shape =  torch.Size([32, 10, 1000])
dec state shape  =  torch.Size([2, 32, 16])


### Seq2Seq Translation with Encoder-Decoder 

In [113]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec, *args):
        enc_outputs, enc_state = self.encoder(x_enc, *args)
        dec_init_state = self.decoder.init_state((enc_outputs, enc_state), *args)
        #only returns decoder output, decode state is not used for final pred
        return decoder(x_dec, dec_init_state)[0]

In [115]:
#Test seq 2 seq 
# Test with sample
vocab_size_source, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
vocab_size_target = 2000
batch_size, num_steps = 32, 10

x_enc = torch.randint(0,vocab_size_source,(batch_size, num_steps))
x_dec = torch.randint(0,vocab_size_target,(batch_size, num_steps))

encoder = Encoder(vocab_size_source, embed_size, num_layers, num_hiddens)
decoder = Decoder(vocab_size_target, embed_size, num_hiddens, num_layers)

# enc_output, enc_state = encoder(x)

# decoder_state = decoder.init_state((enc_output, enc_state))

# y, (dec_output, dec_state) = decoder(x, decoder_state)

seq2seq = Seq2Seq(encoder, decoder)

output = seq2seq(x_enc, x_dec)
print('seq2seq output shape = ', output.shape)


seq2seq output shape =  torch.Size([32, 10, 2000])


### Unit Tests

In [8]:
class SpanishDatasetTest(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
    
    def test4(self):
        self.assertEqual('foo', 'foo1')

unittest.main(argv=[''], exit=False)

F...
FAIL: test4 (__main__.SpanishDatasetTest.test4)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/_s/m9kt_szd4qq9brspl9hd44mc0000gn/T/ipykernel_51936/1831953952.py", line 15, in test4
    self.assertEqual('foo', 'foo1')
AssertionError: 'foo' != 'foo1'
- foo
+ foo1
?    +


----------------------------------------------------------------------
Ran 4 tests in 0.003s

FAILED (failures=1)
